Data cleaning, visulalization and feature engineering

In [1]:
import pandas as pd
df = pd.read_csv('train.csv', header=0)
df_test = pd.read_csv('test.csv', header=0)

In [2]:
# df_test.info()
df_test.head(5)
# df_test.describe()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


I observe that the variables Ticket and Cabin PassengeId, Name and Embarcation are irrelevant, therefore I drop them.


In [3]:
df = df.drop(['Cabin', 'Ticket', 'PassengerId', 'Name', 'Embarked'], axis=1)
df_test = df_test.drop(['Cabin', 'Ticket', 'PassengerId', 'Name', 'Embarked'], axis=1)

I change the sex and Embarked into numerical variable

In [4]:
df['Gender'] = df['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
df = df.drop(['Sex'], axis=1)
# df.head(5)

df_test['Gender'] = df_test['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
df_test = df_test.drop(['Sex'], axis=1)

Sum Parch and SibSP into Familiy size
Remove Family size

In [5]:
df['FamilySize'] = df['SibSp'] + df['Parch']
df = df.drop(['SibSp', 'Parch'], axis=1)
# df.head(5)

df_test['FamilySize'] = df_test['SibSp'] + df_test['Parch']
df_test = df_test.drop(['SibSp', 'Parch'], axis=1)

Filling the missing ages

In [6]:
import numpy as np

median_ages = np.zeros((2,3))
median_ages
for i in range(0, 2):
    for j in range(0, 3):
        median_ages[i,j] = df[(df['Gender'] == i) & \
                              (df['Pclass'] == j+1)]['Age'].dropna().median()
        df['AgeFill'] = df['Age']
        df_test['AgeFill'] = df_test['Age']

for i in range(0, 2):
    for j in range(0, 3):
        df.loc[ (df.Age.isnull()) & (df.Gender == i) & (df.Pclass == j+1),\
                'AgeFill'] = median_ages[i, j]
        df_test.loc[ (df_test.Age.isnull()) & (df_test.Gender == i) & (df_test.Pclass == j+1),\
                'AgeFill'] = median_ages[i, j]
# df[ df['Age'].isnull() ][['Gender','Pclass','Age','AgeFill']].head(10)

In [7]:
df = df.drop(['Age'], axis=1)

df_test = df_test.drop(['Age'], axis=1)
# df.head(10)

Filling the missing fare in the test set

In [8]:
df_test['FareFill'] = df_test['Fare']
df['FareFill'] = df['Fare']

Fare_median = df_test['FareFill'].dropna().median()
df_test.loc[df_test.Fare.isnull(), ['FareFill']] = Fare_median

df_test = df_test.drop(['Fare'], axis=1)
df = df.drop(['Fare'], axis=1)

df_test.info()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 5 columns):
Pclass        418 non-null int64
Gender        418 non-null int64
FamilySize    418 non-null int64
AgeFill       418 non-null float64
FareFill      418 non-null float64
dtypes: float64(2), int64(3)
memory usage: 16.4 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
Survived      891 non-null int64
Pclass        891 non-null int64
Gender        891 non-null int64
FamilySize    891 non-null int64
AgeFill       891 non-null float64
FareFill      891 non-null float64
dtypes: float64(2), int64(4)
memory usage: 41.8 KB


We add the age class feature

In [9]:
df['Age*Class'] = df.AgeFill * df.Pclass
df_test['Age*Class'] = df_test.AgeFill * df_test.Pclass
# df.head(5)

In [10]:
# df.dtypes
train_data = df.values
test_data = df_test.values
# train_data

Test set preprocessing

In [11]:
df.head(2)

,Survived,Pclass,Gender,FamilySize,AgeFill,FareFill,Age*Class
0,0,3,1,1,22.0,7.2500,66.0
1,1,1,0,1,38.0,71.2833,38.0


In [12]:
df_test.head(2)
train_data[0]

array([  0.  ,   3.  ,   1.  ,   1.  ,  22.  ,   7.25,  66.  ])

Random forest

In [46]:
X_train = train_data[0::, 1::]
X_test = test_data[0::, 0::]
Y = train_data[0::, 0]

In [13]:
# Import the random forest package
from sklearn.ensemble import RandomForestClassifier 

# Create the random forest object which will include all the parameters
# for the fit
forest = RandomForestClassifier(n_estimators = 100)

# Fit the training data to the Survived labels and create the decision trees
forest = forest.fit(train_data[0::, 1::],train_data[0::, 0])

# Take the same decision trees and run it on the test data
output = forest.predict(test_data[0::, 0::])

# Score is 0.70

In [47]:
# Logistic regression

from sklearn import linear_model

clf = linear_model.LogisticRegression(C=1e5)
clf = clf.fit(X_train, Y)
output = clf.predict(X_test)

# Score is 0.75

In [48]:
output = output.astype(int)
passenger_id = np.arange(892, 1310, 1)
np.savetxt("prediction.csv", output, delimiter=",")
prediction = np.vstack((passenger_id, output)).transpose()
names = ['PassengerId', 'Survived']
df = pd.DataFrame(prediction, index=passenger_id, columns=names)
df.head(3)
df.to_csv('prediction.csv', index=False, header=True, sep=',')